# 加载库文件

In [ ]:
import json
import math
import pandas as pd
from tqdm import tqdm
import numpy as np
import string
from torch.utils.data.dataset import Dataset
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn import metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error
from torch.utils import data
from sklearn.model_selection import train_test_split
import logging
import datetime
import os
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import r2_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
import torch.optim as optim
import time
from torch import Tensor
import torch.utils.data as Data
import random
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn import manifold
from sklearn.preprocessing import StandardScaler
from IPython.display import display
from sklearn.cluster import KMeans,AgglomerativeClustering
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

def set_seed(seed=2022):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
set_seed(2022)


# 数据加载

## 课程与知识结构

In [ ]:
course = pd.read_pickle('data/select_data/course.pkl')[['id','name','about','resource']]
ALL_course_id = course['id'].tolist()
course.index = list(course['id'])

### 知识结构抽取

In [ ]:
# 调整知识点
def delete_test(one_kp):
    if '作业' in one_kp:
        one_kp = one_kp.split('作业')[0].strip()
        if one_kp != '':
            # 去除’-‘
            for i in range(10):
                if one_kp[-1] == '-':
                    one_kp = one_kp[:-1].strip()
                else:
                    break
            return one_kp
        else:
            return '作业'
    else:
        return one_kp.strip()
resource = []
for in_x, one_course in course.iterrows():
    one_c_id = one_course['id']
    one_c_name = one_course['name']
    one_re_list = one_course['resource']
    one_c_735164 = []
    for one_re in one_re_list:
        one_tit = one_re['titles']
        one_re_id = one_re['resource_id'].split('_')
        re_type = one_re_id[0]
        kp_1 = one_tit[0]
        kp_2 = one_tit[1]
        kp_3 = one_tit[2]
        if kp_2 is None:
            if kp_3 == '作业':
                kp_2 = kp_1
            else:
                kp_2 = kp_3
        if kp_3 == 'Video' or kp_3 == 'video' or kp_3 == '作业':
            kp_3 = kp_2
        kp_1 = delete_test(kp_1)
        kp_2 = delete_test(kp_2)
        kp_3 = delete_test(kp_3)
        if one_c_id in [746997,845950]:
            if '--' in kp_3 and '牛顿' not in kp_3:
                kp_3 = kp_3.split('--')[1]
            if '--' in kp_2:
                kp_2 = kp_2.split('--')[1]
        if one_c_id == 1756063:
            if 'Exercise' in kp_2:
                kp_2 = kp_1
            kp_3 = kp_2
        if one_c_id == 735164:
            if re_type == 'Ex':
                search_str = kp_1+'#####'+kp_2
                for one_oo in one_c_735164:
                    if search_str in one_oo:
                        kp_2 = one_oo.split('#####')[1]
                        kp_3 = one_oo.split('#####')[1]
                        break
            else:
                one_c_735164.append(kp_1+'#####'+kp_2)
        if kp_3 in kp_2:
            kp_3 = kp_2       
        one_re_content = {'course_id':one_c_id,'re_id':int(one_re_id[1]),'re_type':re_type,
                          'title_old':one_tit,
                          'kp_1':kp_1,'kp_2':kp_2,'kp_3':kp_3,
                         'chapter':one_re['chapter'],'c_name':one_c_name}
        resource.append(one_re_content)
resource = pd.DataFrame(resource)
resource = pd.read_csv('data/resource_good.csv')

In [ ]:
ALL_EXM_id = list(resource[resource['re_type']=='Ex']['re_id'])
ALL_VIDEO_id = list(resource[resource['re_type']=='V']['re_id'])
print(len(ALL_EXM_id),'exms')
print(len(ALL_VIDEO_id),'video')
resource['kp_0'] = list(resource['c_name'])
ALL_kp_name = []
ALL_kp_name.extend(list(resource['kp_0']))# kp_0
ALL_kp_name.extend(list(resource['kp_1']))
ALL_kp_name.extend(list(resource['kp_2']))
ALL_kp_name.extend(list(resource['kp_3']))
ALL_kp_name = list(set(ALL_kp_name))
print('kp_num:',len(ALL_kp_name))
kp_name_2_id = dict(zip(ALL_kp_name,list(range(len(ALL_kp_name)))))
kp_id_2_name = dict(zip(list(range(len(ALL_kp_name))),ALL_kp_name))
resource['kp_id'] = resource['kp_3'].map(kp_name_2_id)
resource['kp_0_id'] = resource['kp_0'].map(kp_name_2_id)
resource['kp_1_id'] = resource['kp_1'].map(kp_name_2_id)
resource['kp_2_id'] = resource['kp_2'].map(kp_name_2_id)
resource['kp_3_id'] = resource['kp_3'].map(kp_name_2_id)
resource.head()

In [ ]:
# 抽取关系
ALL_Ex_data = resource[resource['re_type']=='Ex'].copy()
ALL_Video_data = resource[resource['re_type']=='V'].copy()
# 练习与课程关系
ex_2_course = dict(zip(ALL_Ex_data['re_id'],ALL_Ex_data['course_id']))
# 练习与知识点关系
ex_2_kp = dict(zip(ALL_Ex_data['re_id'],ALL_Ex_data['kp_id']))
# 视频与课程关系
video_2_course = dict(zip(ALL_Video_data['re_id'],ALL_Video_data['course_id']))
# 视频与知识点关系
video_2_kp = dict(zip(ALL_Video_data['re_id'],ALL_Video_data['kp_id']))

## 学生作答数据

In [ ]:
user_question = pd.read_pickle('data/select_data/user_question.pkl')
user_question = user_question[user_question['course_id'].isin(ALL_course_id)]
user_question['submit_time'] = pd.to_datetime(user_question['submit_time'])
user_question['kp_id'] = user_question['ex_id'].map(ex_2_kp)
user_question = user_question.sort_values(by=['user_id', 'submit_time'])  # 确保数据按学生和时间排序
user_question['use_time'] = user_question.groupby('user_id')['submit_time'].diff()
user_question['use_time'] = user_question['use_time'].dt.total_seconds()
# 去掉时间过长的
user_question['use_time'] = user_question['use_time']*(user_question['use_time']<600)
user_question['use_time'] = user_question['use_time'].replace(np.NAN,0)
mean_time_diff = user_question.groupby('user_id')['use_time'].mean()
mean_time_diff = dict(zip(mean_time_diff.index,mean_time_diff))
def get_mean(one_row):
    if one_row['use_time'] == 0:
        return mean_time_diff[one_row['user_id']]
    else:
        return one_row['use_time']
user_question['use_time'] = user_question.apply(get_mean,axis=1)
user_question.head(10)

## 计算成绩数据

In [ ]:
# 每个人每门课程的成绩
studentInfo = user_question.groupby(['user_id','course_id'])['is_correct'].mean().reset_index()
studentInfo = studentInfo.drop_duplicates(['user_id'])
stu_course_q_num = user_question.groupby(['user_id','course_id'])['problem_id'].count().reset_index()
stu_course_q_num = stu_course_q_num.drop_duplicates(['user_id'])
stu_2_q_num = dict(zip(list(stu_course_q_num['user_id']),list(stu_course_q_num['problem_id'])))
studentInfo['q_num'] = studentInfo['user_id'].map(stu_2_q_num)
is_pass = (studentInfo['is_correct']>0.95)&(studentInfo['q_num']>90)
print('通过率：',sum(is_pass)/len(studentInfo))
studentInfo['is_pass'] = is_pass
studentInfo['is_pass'] = studentInfo['is_pass'].astype(int)
ALL_STU_ID = list(studentInfo['user_id'])
studentInfo.columns = ['stu_id','course_id','rate_pass','q_num','final_result']

## 学生视频数据

In [ ]:
user_video = pd.read_pickle('data/select_data/user_video.pkl')
user_video = user_video[user_video['course_id'].isin(ALL_course_id)]
user_video = user_video[user_video['user_id'].isin(ALL_STU_ID)]
user_video = user_video.sort_values(by='ts')
user_video['kp_id'] = user_video['video_id'].map(video_2_kp)
user_video['use_time'] = abs(user_video['end_point'] - user_video['start_point'])
user_video.head()

## 用户评论数据

In [ ]:
user_comment = pd.read_pickle('data/select_data/user_comment.pkl')
user_comment = user_comment[user_comment['user_id'].isin(ALL_STU_ID)]
user_comment['create_time'] = pd.to_datetime(user_comment['create_time'])
user_comment = user_comment.sort_values(by='create_time')
user_comment.head()

## 学习行为聚合

In [ ]:
studentInfo['stu_id_index'] = list(studentInfo['stu_id'])

ALL_STU_Start = {}
# ['date', 'sum_click', 'activity_type', 'stu_id', 'stu_id_index']
studentVle = []
comm_col = ['stu_id','id_site','kp_id','course_id','ts']
for one_stu_id in tqdm(studentInfo['stu_id']):
    one_q_data = user_question[user_question['user_id']==one_stu_id].copy()
    one_q_data = one_q_data[['user_id','ex_id','kp_id','course_id','submit_time']]
    one_q_data.columns = comm_col
    one_q_data['activity_type'] = 'question'
    one_q_data['sum_click'] = 1
    
    one_video_data = user_video[user_video['user_id']==one_stu_id].copy()
    one_video_data = one_video_data[['user_id','video_id','kp_id','course_id','ts']]
    one_video_data.columns = comm_col
    one_video_data['activity_type'] = 'video'
    one_video_data['sum_click'] = 1
    
    one_comment_data = user_comment[user_comment['user_id']==one_stu_id].copy()
    one_comment_data = one_comment_data[['user_id','id','create_time']]
    one_comment_data.columns = ['stu_id','id_site','ts']
    one_comment_data['activity_type'] = 'comment'
    one_comment_data['sum_click'] = 1
    one_stu_data = pd.concat([one_q_data, one_video_data, one_comment_data], axis=0)
    one_stu_data = one_stu_data.sort_values(by='ts').reset_index(drop=True)
    
    ALL_STU_Start[one_stu_id] = one_stu_data['ts'][0]
    one_stu_data['date'] = one_stu_data['ts'] - one_stu_data['ts'][0]
    one_stu_data['date'] = one_stu_data['date'].dt.days + 1 
    one_stu_data['stu_id_index'] = list(one_stu_data['stu_id'])
    studentVle.append(one_stu_data)
studentVle = pd.concat(studentVle, axis=0)
studentVle = studentVle.sort_values(by='ts').reset_index(drop=True)
studentVle.head()

In [ ]:
# 每个学生，每个视频看的总时长
user_video_time_sum = user_video.groupby(['user_id','video_id'])['use_time'].sum().reset_index()
# 每个学生看每个视频的平均时长
user_video_time_mean = user_video_time_sum.groupby('user_id')['use_time'].mean()
# 每个视频的平均观看时长
video_time_mean = user_video_time_sum.groupby('video_id')['use_time'].mean()

In [ ]:
# 更新资源信息
resource['video_t_mean'] = resource['re_id'].map(dict(video_time_mean))
studentInfo['video_t_mean'] = studentInfo['stu_id'].map(dict(user_video_time_mean))
studentInfo

# 危险学生预测

In [ ]:
def get_count_data(one_set):
    # 获取次数
    all_stu_data = []
    one_set.index = list(one_set['stu_id_index'])
    one_set_stu = one_set['stu_id_index'].value_counts()
    one_stu_data_0 = dict(zip(ALL_ACT_TYPE,[0]*len(ALL_ACT_TYPE)))
    for one_stu in ALL_STU_ID:
        one_stu_data = one_stu_data_0.copy()
        if one_stu in one_set_stu.index:
            one_stu_set = one_set.loc[one_stu]
            if one_set_stu[one_stu] ==1:
                one_stu_data[one_stu_set['activity_type']] = one_stu_set['sum_click']
            else:
                one_stu_data.update(dict(one_stu_set.groupby('activity_type').sum()['sum_click']))
        all_stu_data.append(one_stu_data)
    one_re = pd.DataFrame(all_stu_data,index=ALL_STU_ID)
    return one_re

def get_tf_idf(count_data):
    # 获取相应的tf-idf值
    all_stu_len = sum(count_data.sum(axis=1)!=0)
    all_idf = {}
    for one_act_type in ALL_ACT_TYPE:
        one_num = sum(count_data[one_act_type]!=0)
        all_idf[one_act_type] = math.log(all_stu_len/(one_num+1),10)
    all_stu_data = []
    one_tf_0 = dict(zip(ALL_ACT_TYPE,[0]*len(ALL_ACT_TYPE)))
    for one_stu in ALL_STU_ID:
        one_stu_data = count_data.loc[one_stu]
        one_sum = sum(one_stu_data)
        if one_sum!=0:
            one_tf = dict(one_stu_data/one_sum) # 字典
        else:
            one_tf = one_tf_0.copy() # 字典
        one_tf_idf = [one_tf[one_a]*all_idf[one_a] for one_a in ALL_ACT_TYPE]
        one_tf_idf = dict(zip(ALL_ACT_TYPE,one_tf_idf))
        all_stu_data.append(one_tf_idf)
    one_re = pd.DataFrame(all_stu_data,index=ALL_STU_ID)
    return one_re

def save_data(stpe_day,is_single,is_count,step_num,one_data_set):
    data_path = f'data/risk_stu/{STEP_DAY}/{is_single}/{is_count}'
    os.makedirs(data_path, exist_ok=True)
    flie_path = f'data/risk_stu/{STEP_DAY}/{is_single}/{is_count}/{one_step}.pkl'
    data_set = one_data_set.copy()
    data_set['stu_id_index'] = ALL_STU_ID
    data_set.to_pickle(flie_path)
    
def performance_risk(ground_truth, prediction):
    auc = metrics.roc_auc_score(ground_truth, prediction) * 100
    # 转成0，1
    prediction = np.round(prediction)
    f1 = metrics.f1_score(ground_truth, prediction) * 100
    recall = metrics.recall_score(ground_truth, prediction) * 100
    precision = metrics.precision_score(ground_truth, prediction) * 100
    acc = metrics.accuracy_score(ground_truth, prediction) * 100
    rmse = np.sqrt(mean_squared_error(ground_truth,prediction))
    one_re_t = [auc, f1, recall, precision, acc,rmse]
    re_col = ['Auc', 'F1', 'Recall', 'Precision', 'Acc','Rmse']
    one_re_dict = dict(zip(re_col,one_re_t))
    return one_re_dict

class LSTM_DataSet(Dataset):
    def __init__(self,all_stu_data,week_need):
        self.all_stu_data = all_stu_data
        self.week_need = week_need
        
    def __len__(self):
        return len(self.all_stu_data)

    def __getitem__(self, index):
        one_data = self.all_stu_data[index]
        x = torch.FloatTensor(one_data['week_data'][:self.week_need])
        comm_data = torch.FloatTensor(one_data['common_data'])
        y = torch.as_tensor(one_data['final_result'], dtype=torch.float)
        return x,y,comm_data

class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim,is_info,info_len,model_name):
        super(LSTMModel, self).__init__()        
        # 定义 LSTM 层
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.is_info = is_info
        self.info_len = info_len
        self.model_name = model_name
        predict_dim = hidden_dim
        if model_name == 'LSTM':
            self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True) 
        elif model_name == 'BiLSTM':
            predict_dim += hidden_dim
            self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True, bidirectional=True) 
        elif model_name == 'RNN':
            self.lstm = nn.RNN(input_dim, hidden_dim, layer_dim, batch_first=True)
        # 定义全连接层
        if is_info:
            predict_dim += hidden_dim
            self.info_layer = nn.Sequential(nn.Linear(self.info_len, hidden_dim),
                                             nn.ReLU(True),
                                             nn.Linear(hidden_dim, hidden_dim))     
        self.class_classifier = nn.Sequential(nn.Linear(predict_dim, predict_dim),
                                             nn.ReLU(True),
                                             nn.Linear(predict_dim, 1),
                                             nn.Sigmoid())
    def forward(self, x,common_data):
        if self.model_name == 'LSTM':
            hidden_len = self.layer_dim
            h0 = torch.zeros(hidden_len, x.size(0), self.hidden_dim).to(DEVICE)
            c0 = torch.zeros(hidden_len, x.size(0), self.hidden_dim).to(DEVICE)
            out, (hn, cn) = self.lstm(x, (h0, c0))
        elif self.model_name == 'BiLSTM':
            hidden_len = self.layer_dim *2
            h0 = torch.zeros(hidden_len, x.size(0), self.hidden_dim).to(DEVICE)
            c0 = torch.zeros(hidden_len, x.size(0), self.hidden_dim).to(DEVICE)
            out, (hn, cn) = self.lstm(x, (h0, c0))
        elif self.model_name == 'RNN':
            hidden_len = self.layer_dim
            h0 = torch.zeros(hidden_len, x.size(0), self.hidden_dim).to(DEVICE)
            out, _ = self.lstm(x, h0)
        out = out[:, -1, :]
        if self.is_info:
            out = torch.cat([out, self.info_layer(common_data)], dim=1)
        out = self.class_classifier(out)
        out = out.squeeze()
        return out

def get_stand_data(one_data_set,use_info):
    scaler = StandardScaler()
    need_data = one_data_set[use_info]
    df_standardized = pd.DataFrame(scaler.fit_transform(need_data), columns=use_info)
    df_standardized['stu_id_index'] = list(one_data_set['stu_id_index'])
    df_standardized.index = list(df_standardized['stu_id_index'])
    return df_standardized

def get_time_data(need_stu_id,data_path,step_num,stu_info_set,stu_info,act_type,is_front,is_stand,is_merge,count_type):
    all_week_data = []
    for one_week in range(1,step_num+1):
        one_data = pd.read_pickle(f'{data_path}/{one_week}.pkl')
        if is_stand:
            one_data = get_stand_data(one_data,act_type)
        all_week_data.append(one_data)
    if is_merge:
        a_conut = 'count' if count_type == 'tf' else 'tf'
        new_path = data_path.replace(count_type,a_conut)
        all_week_data_a = []
        for one_week in range(1,step_num+1):
            one_data = pd.read_pickle(f'{new_path}/{one_week}.pkl')
            if is_stand:
                one_data = get_stand_data(one_data,act_type)
            all_week_data_a.append(one_data)
    all_stu_data = []
    for one_stu in tqdm(need_stu_id,desc='read_data'):
        one_stu_data = {'final_result':stu_info_set.loc[one_stu]['final_result'],'week_data':[],'stu_id':one_stu}
        one_stu_data['common_data'] = list(stu_info_set.loc[one_stu][stu_info])
        for one_week in range(step_num):
            one_week_data = list(all_week_data[one_week].loc[one_stu][act_type])
            if is_merge:
                one_week_data.extend(list(all_week_data_a[one_week].loc[one_stu][act_type]))
            if is_front:
                one_week_data.extend(one_stu_data['common_data'].copy())
            one_stu_data['week_data'].append(one_week_data)
        all_stu_data.append(one_stu_data)
    return all_stu_data

## 数据整合

In [ ]:
ALL_ACT_TYPE = ['video','comment','question']
STEP_DAY = 7
STEP_NUM = max(studentVle['date'])//STEP_DAY
print('最大周数',STEP_NUM)
day_path = f'data/risk_stu/{STEP_DAY}'
print("开始生成数据")
one_step_conut_add = []
for one_step in tqdm(range(1,STEP_NUM+1)):
    # 获取每个周期的计数数据
    one_step_set = studentVle[(studentVle['date']>one_step*(STEP_DAY-1)) & (studentVle['date']<=one_step*STEP_DAY)].copy()
    one_step_count_single = get_count_data(one_step_set)
    # 获取累积周期的计数数据
    if one_step == 1:
        one_step_conut_add = one_step_count_single.copy()
    else:
        one_step_conut_add = one_step_conut_add+one_step_count_single
    # 获取每个周期的TF数据
    one_step_tf_single = get_tf_idf(one_step_count_single)
    # 获取累积周期的TF数据
    one_step_tf_add = get_tf_idf(one_step_conut_add)
    save_data(STEP_DAY,'single','count',one_step,one_step_count_single)
    save_data(STEP_DAY,'add','count',one_step,one_step_conut_add)
    save_data(STEP_DAY,'single','tf',one_step,one_step_tf_single)
    save_data(STEP_DAY,'add','tf',one_step,one_step_tf_add)

## 模型训练

In [ ]:
studentInfo.index = list(studentInfo['stu_id_index'])
studentInfo.head()
STU_INFO = ['stu_id_index']
# 参数设置
EPOCH_NUM = 50
BATCH_SIZE = 32
DEVICE = 'cpu'
STEP_DAY = 7 # 每隔几天采集一次数据
STEP_NUM = 15# 运行的周数
IS_COUNT = 1 # 是否使用conut值
IS_ADD = 1 # 是否采用累加数据
USE_INFO = 0 # 是否使用学生信息
IS_MERGE = 1 # 是否把tf和count合并到一起
IS_FRONT = 0 # 是否在LSTM里面加入学生信息
IS_STAND = 1 # 是否归一化
COUNT_TYPE = 'count' if IS_COUNT else 'tf'
ADD_TYPE = 'add'if IS_ADD else 'single'
ALL_ML_MODEL = ['RFC','KNN','DT','GNB','SVM','GBT','LR']
DEEP_MODEL = ['LSTM','RNN']
MODEL_NAME = 'RFC'
config = {
    'Model': MODEL_NAME,
    'Add_type': ADD_TYPE,
    'Count_type': COUNT_TYPE,
    'USE_INFO': USE_INFO,
    'BATCH_SIZE': BATCH_SIZE,
    'IS_FRONT': IS_FRONT,
    'IS_STAND': IS_STAND,
    'IS_MERGE': IS_MERGE
}
logging.shutdown()
time_now = datetime.datetime.now()
now_str = time_now.strftime('%Y-%m-%d-%H-%M-%S')
log_dir = f'log/risk_stu/{STEP_DAY}/{MODEL_NAME}/{COUNT_TYPE}'
log_file_name = f'{log_dir}/{ADD_TYPE}-{now_str}.log'
os.makedirs(log_dir, exist_ok=True)
logging.basicConfig(level=logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(message)s')
file_logger = logging.getLogger(log_file_name)
file_handler = logging.FileHandler(log_file_name, mode='a', encoding='utf-8')
file_handler.setFormatter(formatter)
file_logger.addHandler(file_handler)
file_logger.info(f"{config}")
TRAIN_DATA, TEST_DATA = train_test_split(studentInfo,test_size=0.2, random_state=42, stratify=studentInfo['final_result'])
TRAIN_DATA_STU = list(TRAIN_DATA['stu_id'])
TEST_DATA_STU = list(TEST_DATA['stu_id'])

TRAIN_DATA = list(TRAIN_DATA['stu_id_index'])
print('TRAIN_DATA len',len(TRAIN_DATA))
TEST_DATA = list(TEST_DATA['stu_id_index'])
print('TEST_DATA len',len(TEST_DATA))
TEST_DATA_raw = studentInfo[studentInfo['stu_id_index'].isin(TEST_DATA)].copy()
ALL_STEP_RE = []
for ONE_STEP in range(1,STEP_NUM+1): 
    DATA_PATH = f'data/risk_stu/{STEP_DAY}/{ADD_TYPE}/{COUNT_TYPE}'
    best_one_re_dict = {}
    if MODEL_NAME in ALL_ML_MODEL:
        one_data = pd.read_pickle(f'{DATA_PATH}/{ONE_STEP}.pkl')
        ML_data = pd.concat([studentInfo,one_data],axis=1)
        ML_col = ALL_ACT_TYPE.copy()
        ML_col.extend(STU_INFO)
        X_train,y_train = ML_data.loc[TRAIN_DATA][ML_col].values,ML_data.loc[TRAIN_DATA]['final_result']
        X_test,y_test = ML_data.loc[TEST_DATA][ML_col].values,ML_data.loc[TEST_DATA]['final_result']
        if MODEL_NAME=='RFC':
            model = RandomForestClassifier(n_estimators=40,max_depth=18,random_state=10)
        if MODEL_NAME=='KNN':
            model = KNeighborsClassifier()
        if MODEL_NAME=='DT':
            model = DecisionTreeClassifier(max_depth=7,random_state=2023)
        if MODEL_NAME=='GNB':
            model =  GaussianNB()
        if MODEL_NAME=='SVM':
            model = SVC(kernel='rbf',probability=True)
        if MODEL_NAME=='GBT':
            model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
        if MODEL_NAME=='LR':
            model = LogisticRegression(max_iter=50)
        model = model.fit(X_train,y_train)
        y_train_predict = model.predict_proba(X_train)[:,1]
        one_re_dict = performance_risk(y_train,y_train_predict)
        file_logger.info(f"STEP {ONE_STEP} Train: {one_re_dict}")
        y_test_predict = model.predict_proba(X_test)[:,1]
        best_one_re_dict = performance_risk(y_test,y_test_predict)
        TEST_DATA_raw[f'week_{ONE_STEP}'] = list(y_test_predict)
        file_logger.info(f"STEP {ONE_STEP} Test: {best_one_re_dict}")
    if MODEL_NAME in ['LSTM','RNN']:
        file_logger.info(f'Start Week :{ONE_STEP}')
        data_params = {
            'data_path': DATA_PATH,
            'step_num': ONE_STEP,
            'stu_info_set': studentInfo,
            'stu_info': STU_INFO,
            'act_type': ALL_ACT_TYPE,
            'is_front': IS_FRONT,
            'is_stand': IS_STAND,
            'is_merge': IS_MERGE,
            'count_type': COUNT_TYPE
        }
        train_data = get_time_data(TRAIN_DATA,**data_params)
        train_data = LSTM_DataSet(train_data,ONE_STEP)
        test_data = get_time_data(TEST_DATA,**data_params)
        test_data = LSTM_DataSet(test_data,ONE_STEP)
        train_laoder = data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
        test_laoder = data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True)
        input_dim = len(ALL_ACT_TYPE)
        if IS_FRONT:
            input_dim += len(STU_INFO)# 输入维度
        if IS_MERGE:
            input_dim += len(ALL_ACT_TYPE)
        hidden_dim = 8  # 隐藏状态维度
        layer_dim = 1  # LSTM层数
        output_dim = 1  # 输出维度
        info_len = len(STU_INFO)
        LR = 0.00001
        model = LSTMModel(input_dim, hidden_dim, layer_dim, output_dim,USE_INFO,info_len,MODEL_NAME)
#         criterion = nn.MSELoss()
        criterion = nn.BCEWithLogitsLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=LR, betas=(0.9, 0.999), eps=1e-8)
        file_logger.info(f'input_dim:{input_dim}, hidden_dim:{hidden_dim}, layer_dim:{hidden_dim}, output_dim:{output_dim},info_len:{info_len}')
        model.to(DEVICE)
        best_auc = 0
        best_ecoch = 0
        # 训练模型
        for one_epoch in range(EPOCH_NUM):
            model.train()
            ground_truth = torch.Tensor([])
            prediction = torch.Tensor([])
            for inputs, labels,common_data in tqdm(train_laoder, desc=f'Week {ONE_STEP} Train-{one_epoch}:',):
                inputs.to(DEVICE)
                labels.to(DEVICE)
                common_data.to(DEVICE)
                optimizer.zero_grad()
                outputs = model(inputs,common_data)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                prediction = torch.cat([prediction, outputs.to('cpu')])
                ground_truth = torch.cat([ground_truth, labels.to('cpu')])
            one_re_dict = performance_risk(ground_truth.detach().numpy(), prediction.detach().numpy())
            file_logger.info(f'Week {ONE_STEP} Train-{one_epoch} Preformance:{one_re_dict}')
            model.eval()
            ground_truth = torch.Tensor([])
            prediction = torch.Tensor([])
            for inputs, labels,common_data in tqdm(test_laoder, desc=f'Week {ONE_STEP} Test-{one_epoch}::',):
                inputs.to(DEVICE)
                common_data.to(DEVICE)
                with torch.no_grad():
                    outputs = model(inputs,common_data)
                    prediction = torch.cat([prediction, outputs.to('cpu')])
                    ground_truth = torch.cat([ground_truth, labels.to('cpu')])
            one_re_dict = performance_risk(ground_truth.detach().numpy(), prediction.detach().numpy())
            file_logger.info(f'Week {ONE_STEP}  Test-{one_epoch}Preformance:{one_re_dict}')
            if one_re_dict['Auc'] > best_auc:
                best_auc = one_re_dict['Auc']
                best_ecoch = one_epoch
                file_logger.info(f'Best_acu in Valid:【{best_auc}】 at epoch 【{one_epoch}】')
                best_one_re_dict = one_re_dict.copy()
                TEST_DATA_raw[f'week_{ONE_STEP}'] = list(prediction.detach().numpy())
            if one_epoch-best_ecoch>10:
                print('Train stop')
                break
    best_one_re_dict['STEP'] = ONE_STEP
    file_logger.info(f'Test【{ONE_STEP}】:{best_one_re_dict}')
    ALL_STEP_RE.append(best_one_re_dict)
file_logger.info(f"{config}")
ALL_STEP_RE_set = pd.DataFrame(ALL_STEP_RE)
print(ALL_STEP_RE_set)
ALL_STEP_RE_set.to_csv(f'{log_dir}/{MODEL_NAME}-{ADD_TYPE}-{now_str}.csv')
logging.shutdown()
ALL_STEP_RE_set[['Auc', 'F1', 'Recall', 'Precision', 'Acc']].plot()

# 知识追踪

In [ ]:
# 读取数据，划分数据
class MyDataset_kt(Dataset):
    def __init__(self, X):
        self.all_item = X[USE_ITEM[:-1]].values  
        self.all_label = X['is_right'].values  
        self.all_index = list(X['index'])
        del X

    def __getitem__(self, index):
        item = torch.as_tensor(self.all_item[index], dtype=torch.float)  
        label = torch.as_tensor(self.all_label[index], dtype=torch.float)
        one_index = int(self.all_index[index]) 
        return item, label, one_index

    def __len__(self):
        # 返回数据集大小
        return len(self.all_item)


def standard_scaler_kt(DF):
    st_sc = StandardScaler()
    scaler_data = DF[USE_ITEM[:-1]]
    add_data = DF[['is_right','index']]
    scaler_data_x = st_sc.fit_transform(scaler_data)
    scaler_data_ed = pd.DataFrame(scaler_data_x, columns=USE_ITEM[:-1])
    scaler_data_ed = scaler_data_ed.join(add_data)
    return scaler_data_ed

class MyModel(nn.Module):
    def __init__(self, input_dim):
        super(MyModel, self).__init__()
        self.class_classifier = nn.Sequential(nn.Linear(input_dim, input_dim*2),
                                             nn.ReLU(True),
                                             nn.Linear(input_dim*2, input_dim*4),
                                             nn.ReLU(True),
                                             nn.Linear(input_dim*4, input_dim*2),
                                             nn.ReLU(True),
                                             nn.Linear(input_dim*2, 10),
                                             nn.ReLU(True),
                                             nn.Linear(10, 1),
                                             nn.Sigmoid())
    def forward(self, item):
        out = self.class_classifier(item)
        out = out.squeeze()
        return out
    

# 计算性能
def performance_kt(ground_truth, prediction):
    ground_truth = ground_truth.to('cpu')
    prediction = prediction.to('cpu')
    ground_truth = ground_truth.detach().numpy()
    prediction = prediction.detach().numpy()
    auc = metrics.roc_auc_score(ground_truth, prediction) * 100
    # 转成0，1
    prediction = np.round(prediction)
    f1 = metrics.f1_score(ground_truth, prediction) * 100
    recall = metrics.recall_score(ground_truth, prediction) * 100
    precision = metrics.precision_score(ground_truth, prediction) * 100
    acc = metrics.accuracy_score(ground_truth, prediction) * 100
    rmse = np.sqrt(mean_squared_error(ground_truth,prediction))
    return auc, f1, recall, precision, acc,rmse

def __load_model__(ckpt):
    if os.path.isfile(ckpt):
        checkpoint = torch.load(ckpt)
        print("Successfully loaded checkpoint '%s'" % ckpt)
        return checkpoint
    else:
        raise Exception("No checkpoint found at '%s'" % ckpt)

## 数据整合与侧面信息提取

In [ ]:
tqdm_ncols = 80
def updata_data(old_data, one_key, one_row_data):
    if one_key not in old_data.keys():
        old_data[one_key] = [
            one_row_data['is_right'], one_row_data['use_time'], 1
        ]
        return old_data, [0, 0, 0]
    else:
        one_data = old_data[one_key]
        re_data = [(one_data[0] / one_data[2]) * 100,
                   one_data[1] / one_data[2], one_data[2]]
        one_data[0] += one_row_data['is_right']
        one_data[1] += one_row_data['use_time']
        one_data[2] += 1
        old_data[one_key] = one_data
        return old_data, re_data


def updata_answer_data(old_data, one_key, one_row_data):
    if one_key not in old_data.keys():
        old_data[one_key] = [
            one_row_data['is_right'], one_row_data['use_time'], 1
        ]
        return old_data, [
            one_row_data['is_right'] * 100, one_row_data['use_time'], 1
        ]
    else:
        one_data = old_data[one_key]
        one_data[0] += one_row_data['is_right']
        one_data[1] += one_row_data['use_time']
        one_data[2] += 1
        old_data[one_key] = one_data
        re_data = [(one_data[0] / one_data[2]) * 100,
                   one_data[1] / one_data[2], one_data[2]]
        return old_data, re_data


def get_all_info(all_info, col_name, col_value, new_record_info):
    if col_value not in all_info.keys():
        one_q_data = new_record_info[new_record_info[col_name] == col_value]
        all_info[col_value] = [
            np.mean(one_q_data['is_right']),
            np.mean(one_q_data['use_time']),
            len(one_q_data.index)
        ]
    return all_info


def updata_time_data(old_data, one_key, create_time_now):
    if one_key not in old_data.keys():
        old_data[one_key] = create_time_now
        return old_data, 0
    else:
        old_time = old_data[one_key]
        re_data = create_time_now - old_time
        old_data[one_key] = create_time_now
        return old_data, re_data
    
def updata_passed_data(old_data, one_key, now_index):
    if one_key not in old_data.keys():
        old_data[one_key] = [now_index]
        return old_data, []
    else:
        re_data = old_data[one_key].copy()
        old_data[one_key].append(now_index)
        return old_data, re_data


def get_data(raw_data, stu_kp_data, stu_q_data, kp_data, q_data, q_all_data,
             kp_all_data, stu_kp_data_an, stu_q_data_an, kp_data_an,
             q_data_an,last_kp_time,last_q_time,passed_kp_list,passed_q_list):
    all_data = []
    for i in tqdm(raw_data.index,
                       desc='Process',
                       mininterval=0.2,
                       ncols=tqdm_ncols):
        one_data = []
        one_re_data = raw_data.loc[i]
        stu_id = one_re_data['stu_id']
        kp_id = one_re_data['kp_id']
        q_id = one_re_data['q_id']
        stu_kp = str(int(stu_id)) + '_' + str(int(kp_id))
        one_time = one_re_data['create_time']

        stu_kp_data, one_stu_kp_data = updata_data(
            stu_kp_data,
            stu_kp, one_re_data)
        one_data.extend(one_stu_kp_data)

        # 'stu_q_now_acc', 'stu_q_now_time', 'stu_q_now_len'
        stu_q_data, one_stu_q_data = updata_data(stu_q_data, str(int(stu_id)),
                                                 one_re_data)
        one_data.extend(one_stu_q_data)

        # 'q_now_acc','q_now_use_time','q_now_len'
        q_data, one_q_data = updata_data(q_data, str(int(q_id)), one_re_data)
        one_data.extend(one_q_data)
        
        # 'kp_now_acc','kp_now_use_time','kp_now_len'
        kp_data, one_kp_data = updata_data(kp_data, str(int(kp_id)),
                                           one_re_data)
        one_data.extend(one_kp_data)

        # 'q_all_acc','q_all_use_time','q_all_len'
        q_all_data = get_all_info(q_all_data, 'q_id', q_id, raw_data)
        one_data.extend(q_all_data[q_id])

        # 'kp_all_acc','kp_all_use_time','kp_all_len'
        kp_all_data = get_all_info(kp_all_data, 'kp_id', kp_id, raw_data)
        one_data.extend(kp_all_data[kp_id])

        # 'stu_kp_now_acc_an','stu_kp_now_use_time_an','stu_kp_now_len_an'
        stu_kp_data_an, one_stu_kp_data_an = updata_answer_data(
            stu_kp_data_an,
            str(int(stu_id)) + '_' + str(int(kp_id)), one_re_data)
        one_data.extend(one_stu_kp_data_an)

        # 'stu_q_now_acc_an', 'stu_q_now_time_an', 'stu_q_now_len_an'
        stu_q_data_an, one_stu_q_data_an = updata_answer_data(
            stu_q_data_an, str(int(stu_id)), one_re_data)
        one_data.extend(one_stu_q_data_an)

        # 'q_now_acc_an','q_now_use_time_an','q_now_len_an'
        q_data_an, one_q_data_an = updata_answer_data(q_data_an,
                                                      str(int(q_id)),
                                                      one_re_data)
        one_data.extend(one_q_data_an)
        # 'kp_now_acc_an','kp_now_use_time_an','kp_now_len_an'
        kp_data_an, one_kp_data_an = updata_answer_data(
            kp_data_an, str(int(kp_id)), one_re_data)
        one_data.extend(one_kp_data_an)
        
        # 'last_kp_time_passed','last_q_time_passed'
        last_kp_time,last_kp_time_passed = updata_time_data(last_kp_time,stu_kp, one_time)
        last_q_time,last_q_time_passed = updata_time_data(last_q_time,str(int(stu_id)),one_time)
        one_data.extend([last_kp_time_passed,last_q_time_passed])
        
        # 'passed_kp','passed_q'
        passed_kp_list,passed_kp = updata_passed_data(passed_kp_list,stu_kp,i)
        passed_q_list,passed_q = updata_passed_data(passed_q_list,str(int(stu_id)),i)
        one_data.extend([passed_kp,passed_q])
        
        all_data.append(one_data)
    all_data_2 = np.array(all_data,dtype=object)
    new_col = [
        'stu_kp_now_acc', 'stu_kp_now_use_time', 'stu_kp_now_len',
        'stu_q_now_acc', 'stu_q_now_time', 'stu_q_now_len', 
        'q_now_acc', 'q_now_use_time', 'q_now_len', 
        'kp_now_acc', 'kp_now_use_time','kp_now_len', 
        'q_all_acc', 'q_all_use_time', 'q_all_len', 
        'kp_all_acc','kp_all_use_time', 'kp_all_len', 
        'stu_kp_now_acc_an','stu_kp_now_use_time_an', 'stu_kp_now_len_an', 
        'stu_q_now_acc_an','stu_q_now_time_an', 'stu_q_now_len_an', 
        'q_now_acc_an','q_now_use_time_an', 'q_now_len_an', 
        'kp_now_acc_an','kp_now_use_time_an', 'kp_now_len_an',
        'last_kp_time_passed','last_q_time_passed','passed_kp','passed_q'
    ]
    processed_data = raw_data.copy()
    for j, one_col in enumerate(new_col):
        processed_data[one_col] = all_data_2[:, j]
    return processed_data



data_name = 'MOOC'
processed_data_name = f'data/kt/{data_name}_info.pkl'
print(f'{data_name} processing')
raw_data = user_question.copy()
raw_data.columns = ['q_id', 'stu_id', 'is_right', 'create_time', 'seq_id','course_id', 'kp_id', 'use_time']
print(raw_data.head())
stu_kp_data = {}
stu_q_data = {}
kp_data = {}
q_data = {}
q_all_data = {}
kp_all_data = {}
stu_kp_data_an = {}
stu_q_data_an = {}
kp_data_an = {}
q_data_an = {}
last_kp_time = {}
last_q_time = {}
passed_kp_list = {}
passed_q_list = {}
processed_data= get_data(raw_data, stu_kp_data, stu_q_data, kp_data, q_data, q_all_data,kp_all_data, stu_kp_data_an, stu_q_data_an, kp_data_an, q_data_an,last_kp_time,last_q_time,passed_kp_list,passed_q_list)

processed_data = processed_data.reset_index(drop=True)
def get_sec(one_d):
    if one_d != 0:
        one_d = one_d.total_seconds()
        return one_d
    else:
        return 1
processed_data['last_kp_time_passed'] = processed_data['last_kp_time_passed'].apply(get_sec)
processed_data['last_q_time_passed'] = processed_data['last_q_time_passed'].apply(get_sec)
processed_data.to_pickle(processed_data_name)
print(data_name, 'Side information extraction processing completed！file at：', processed_data_name)
processed_data.head()

## 模型训练

In [ ]:
USE_ITEM = ['stu_kp_now_acc',
            'stu_kp_now_use_time',
            'stu_kp_now_len',
            'stu_q_now_acc',
            'stu_q_now_time',
            'stu_q_now_len',
            'q_now_acc',
            'q_now_use_time',
            'q_now_len',
            'kp_now_acc',
            'kp_now_use_time',
            'kp_now_len',
            'last_kp_time_passed',
            'last_q_time_passed',
            'is_right'
           ]
need_Item = ['stu_id','q_id','kp_id','course_id','seq_id','create_time']
need_Item.extend(USE_ITEM)
DATASET_NAME = 'MOOC'
RAW_DATA = processed_data.copy()[need_Item]
RAW_DATA = RAW_DATA.reset_index()
raw_data_ts = RAW_DATA['create_time'].copy()
RAW_DATA['create_time'] = 0
RAW_DATA = RAW_DATA.astype(np.float32)
RAW_DATA['create_time'] = raw_data_ts
RAW_DATA.index = list(RAW_DATA['stu_id'])
ALL_STU_ID_kt = list(RAW_DATA['stu_id'].value_counts().index)
X_train, X_test = train_test_split(ALL_STU_ID_kt, random_state=2024, test_size=0.2, shuffle=True)
TRAIN_DATA_KT = RAW_DATA.loc[X_train].copy().reset_index(drop=True)
VAIL_DATA_KT = RAW_DATA.loc[X_test].copy().reset_index(drop=True)
BATCH_SIZE = 64
MAX_STEP = 50
LR = 0.0001
trian_data = MyDataset_kt(TRAIN_DATA_KT)
train_data_loader = Data.DataLoader(trian_data, batch_size=BATCH_SIZE, shuffle=True)
vail_data = MyDataset_kt(VAIL_DATA_KT)
vail_data_loader = Data.DataLoader(vail_data, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
# 设置日志文件
MODEL_NAME = 'SIKT'
MODEL_INFO = '测试'
EPOCH_NUM = 20
logging.shutdown()
time_now = datetime.datetime.now()
now_str = time_now.strftime('%Y-%m-%d-%H-%M-%S')
log_dir = f'log/kt'
log_file_name = f'log/kt/{MODEL_NAME}_{now_str}_{MODEL_INFO}.log'
os.makedirs(log_dir, exist_ok=True)
logging.basicConfig(level=logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(message)s')
file_logger = logging.getLogger(log_file_name)
# 向文件输出日志信息
file_handler = logging.FileHandler(log_file_name, mode='a', encoding='utf-8')
file_handler.setFormatter(formatter)
file_logger.addHandler(file_handler)
file_logger.info(f"{config}")
# 模型保存
model_dir = f'log/kt/Checkpoints' 
os.makedirs(model_dir, exist_ok=True)
best_model_path = f'{model_dir}/{MODEL_NAME}_{now_str}.pth.tar'
DEVICE = "cuda:1" if torch.cuda.is_available() else "cpu"
print(DEVICE)
model = MyModel(input_dim=len(USE_ITEM)-1)
loss_func = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)
file_logger.info(f'best_model_path:{best_model_path}')
best_auc = 0
model.to(DEVICE)
for one_epoch in range(EPOCH_NUM):
    ground_truth = torch.tensor([])
    prediction = torch.tensor([])
    model.train()
    for batch in tqdm(train_data_loader, desc='Training:    ', mininterval=2):
        item, label_batch,one_indexs = batch
        item, label_batch = item.to(DEVICE), label_batch.to(DEVICE)
        optimizer.zero_grad()
        pred_batch = model(item)
        loss = loss_func(pred_batch, label_batch)
        loss.backward()
        optimizer.step()
        prediction = torch.cat([prediction, pred_batch.to('cpu')])
        ground_truth = torch.cat([ground_truth, label_batch.to('cpu')])
    auc, f1, recall, precision, acc,rmse  = performance_kt(ground_truth, prediction)
    file_logger.info(f'Train {one_epoch}: auc:{auc},f1:{f1},recall:{recall},precision:{precision},acc:{acc},rmse:{rmse}')
    # 验证
    ground_truth = torch.tensor([])
    prediction = torch.tensor([])
    model.eval()
    for batch in tqdm(vail_data_loader, desc='valid:    ', mininterval=2):
        with torch.no_grad():
            item, label_batch,one_indexs = batch
            item = item.to(DEVICE)
            pred_batch = model(item)
            prediction = torch.cat([prediction, pred_batch.to('cpu')])
            ground_truth = torch.cat([ground_truth, label_batch])
    auc, f1, recall, precision, acc,rmse  = performance_kt(ground_truth, prediction)
    if auc > best_auc:
        best_auc = auc
#         best_model = model
        file_logger.info(f'best auc 【{one_epoch}】')
        torch.save({'state_dict': model.state_dict()}, best_model_path)
    file_logger.info(f'valid {one_epoch}: auc:{auc},f1:{f1},recall:{recall},precision:{precision},acc:{acc},rmse:{rmse}')
logging.shutdown()

## 预测

In [ ]:
risk_stu_have_q = list(set(list(RAW_DATA['stu_id']))&set(ALL_risk_stu))
checkpoint = __load_model__(best_model_path)
model.load_state_dict(checkpoint['state_dict'])
model.eval()
ALL_Pred = []
STU_KP_LEARN = {}
for one_stu in tqdm(risk_stu_have_q):
    TEST_DATA,one_stu_learn_kp,one_stu_no_learn_kp = get_risk_stu(one_stu,stu_risk_week_num[one_stu])
    STU_KP_LEARN[one_stu] = {'learned_kp':one_stu_learn_kp,'unlearned_kp':one_stu_no_learn_kp}
    test_data = MyDataset_kt(TEST_DATA)
    test_data_loader = Data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)
    ground_truth = torch.tensor([])
    prediction = torch.tensor([])
    all_indexs = torch.tensor([])
    for batch in test_data_loader:
        with torch.no_grad():
            item, label_batch,one_indexs = batch
            item = item.to(DEVICE)
            pred_batch = model(item)
            prediction = torch.cat([prediction, pred_batch.to('cpu')])
            ground_truth = torch.cat([ground_truth, label_batch])
            all_indexs = torch.cat([all_indexs, one_indexs])
    all_indexs = all_indexs.tolist()
    prediction = prediction.tolist()
    TEST_DATA['pred_y'] = prediction
    ALL_Pred.append(TEST_DATA)
ALL_Pred = pd.concat(ALL_Pred).reset_index(drop=True)

In [ ]:
STU_KP_master = ALL_Pred.groupby(['stu_id','kp_id'])['pred_y'].mean().rename('master')
STU_KP_master.head()

In [ ]:
STU_KP_master_flat = STU_KP_master.reset_index()
stu_2_course = dict(zip(studentInfo['stu_id'],studentInfo['course_id']))
STU_KP_master_flat['course_id'] = STU_KP_master_flat['stu_id'].map(stu_2_course)
STU_KP_master_flat = STU_KP_master_flat.sort_values('master')
STU_KP_master_flat =STU_KP_master_flat.reset_index(drop=True)
def add_unlearned(one_row):
    one_data = STU_KP_LEARN[one_row['stu_id']]
    if one_row['kp_id'] in one_data['unlearned_kp']:
        return -1
    else:
        return one_row['master']
STU_KP_master_flat['master_un'] = STU_KP_master_flat.apply(add_unlearned,axis=1)
STU_KP_master_flat

# 学习资源推荐

In [ ]:
def generate_uppercase_sequence(length):
    uppercase_letters = string.ascii_uppercase
    sequence = []
    for i in range(length):
        sequence.append(uppercase_letters[i % 26])
    return sequence

def get_seq(one_seq):
    one_real_seq = []
    for one in one_seq:
        one_real_seq.append(event_code_A[one])
    return one_real_seq

def sort_dict(one_dict,reverse=True):
    return dict(sorted(one_dict.items(), key=lambda x: x[1],reverse=True))

def get_dup_seq(one_seq,k):
    one_dup_seq = {}
    for i in range(len(one_seq) - k + 1):
        subseq = one_seq[i:i+k]
        seq_click = 1
        if subseq in one_dup_seq:
            one_dup_seq[subseq] += seq_click
        else:
            one_dup_seq[subseq] = seq_click
    one_dup_seq = sort_dict(one_dup_seq)
    return one_dup_seq

## 数据整合

In [ ]:
N_LIST =  [5,15,25,35,45,55,65,75,85,95]
M = 40
K = 4
LBP_LEN = 4
USE_TF_IDF = True
USE_MODULE_INFO = False
USE_CLUSTER = True
resource_counts_per_student = studentVle.groupby('stu_id')['id_site'].nunique()
average_resources_per_student = resource_counts_per_student.mean()
print(f'平均每个学生学习过的资源个数: {average_resources_per_student}')
all_act_type = list(studentVle['activity_type'].value_counts().index)
event_code = dict(zip(all_act_type,generate_uppercase_sequence(len(all_act_type))))
event_code_A = dict(zip(generate_uppercase_sequence(len(all_act_type)),all_act_type))
studentVle['event_code'] = studentVle['activity_type'].map(event_code)
studentVle.index = list(studentVle['stu_id'])
student_module = studentVle[['course_id','stu_id']].drop_duplicates()
studentInfo_one_hot = pd.get_dummies(student_module,columns=['course_id'])
studentInfo_one_hot = studentInfo_one_hot.groupby('stu_id').sum()
studentInfo_one_hot.index = list(studentInfo_one_hot.index)
# studentInfo_one_hot.head()
studentVle.head()

In [ ]:
all_seq_data = {}
stu_resource = {}
need_stu_have = list(studentVle['stu_id'].value_counts().index)
for one_stu in tqdm(need_stu_have):
    one_stu_data = studentVle.loc[one_stu].copy()
    one_num = dict(one_stu_data['activity_type'].value_counts())
    one_stu_play = one_stu_data[one_stu_data['activity_type'].isin(['video','question'])].copy()
    
    one_stu_res = dict(one_stu_play['id_site'].value_counts())
    stu_resource[one_stu] = one_stu_res
    all_seq_data[one_stu] = list(one_stu_data['event_code'])

all_stu_seq = {}
all_seq_num = {}
for one_stu,one_seq in tqdm(all_seq_data.items()):
    all_stu_seq[one_stu] = {}
    one_dup_seq = get_dup_seq(''.join(one_seq),LBP_LEN)
    all_stu_seq[one_stu] = one_dup_seq
    for one_sub_key in one_dup_seq.keys():
        if one_sub_key in all_seq_num:
            all_seq_num[one_sub_key] += 1
        else:
            all_seq_num[one_sub_key] = 1

if LBP_LEN >2:
    all_seq_num = {k:v for k, v in all_seq_num.items() if v>=100}
    

all_stu_num = len(all_seq_data.keys())
seq_IDF = [math.log(all_stu_num/(one+1),10) for one in list(all_seq_num.values())]
all_seq_IDF = dict(zip(list(all_seq_num.keys()),seq_IDF))
stu_TF_IDF = {}
for key,sub_seq in tqdm(all_stu_seq.items()):
    one_stu_TF_IDF = {}
    seq_all_num = sum(list(sub_seq.values()))
    for sub_k,sub_v in sub_seq.items():
        if sub_k not in all_seq_IDF:
            continue
        one_TF = sub_v/seq_all_num
        one_stu_TF_IDF[sub_k] = all_seq_IDF[sub_k]*one_TF
    stu_TF_IDF[key] = one_stu_TF_IDF

need_feature = dict(zip(list(all_seq_num.keys()),[0]*len(all_seq_num)))
all_feature_values = []
for one_stu,one_TF_IDF in tqdm(stu_TF_IDF.items()):
    one_need_feature = need_feature.copy()
    if USE_TF_IDF:
        one_need_feature.update(one_TF_IDF)
    else:
        one_need_feature.update(all_stu_seq[one_stu])
    all_feature_values.append(one_need_feature)
stu_id_index = list(stu_TF_IDF.keys())
my_data = pd.DataFrame(all_feature_values,index=stu_id_index)
my_data.head()

In [ ]:
if USE_MODULE_INFO:
    my_data = pd.concat([my_data,studentInfo_one_hot.loc[stu_id_index]],join='inner',axis=1)
    my_data.head()
    
if USE_CLUSTER:
    tsne = manifold.TSNE(n_components=2,learning_rate='auto', init='pca', random_state=42).fit_transform(my_data)
    KM_cluster = KMeans(init='k-means++', n_clusters=K,random_state=2023)
    KM_cluster.fit(tsne)
    all_labels = list(KM_cluster.labels_)
    stu_2_cluster = dict(zip(stu_id_index,all_labels))

start_time = time.time()
all_stu_pearson = {}
if USE_CLUSTER:
    cluster_data = my_data.copy()
    cluster_data['cluster'] = all_labels
    nee_col = list(cluster_data.columns[:-1])
    for one_cluster in tqdm(range(K)):
        one_clus_data = cluster_data[cluster_data['cluster']==one_cluster][nee_col]
        all_stu_pearson[one_cluster] = one_clus_data.T.corr(method='pearson')
    display(all_stu_pearson[0].head())
else:
    all_stu_pearson =  my_data.T.corr(method='pearson')
    display(all_stu_pearson.head())
end_time = time.time()
run_time = end_time - start_time
print('run time：',run_time)

In [ ]:
all_stu_sim_resource = {}
for one_stu in tqdm(stu_id_index):
    if USE_CLUSTER:
        one_cluster = stu_2_cluster[one_stu]
        k_person = dict(all_stu_pearson[one_cluster].loc[one_stu].sort_values(ascending=False)[:M+1])
    else:
        k_person = dict(all_stu_pearson.loc[one_stu].sort_values(ascending=False)[:M+1])
    # 去掉自己
    k_person[one_stu] = 10
    del k_person[one_stu]
    one_stu_resource = {}
    for sim_stu,sim in k_person.items():
        one_sim_stu_resource = list(stu_resource[sim_stu].keys())
        for one_resource in one_sim_stu_resource:
            if one_resource not in one_stu_resource:
                one_stu_resource[one_resource] = 0
            one_stu_resource[one_resource] += sim
    all_stu_sim_resource[one_stu] = sorted(one_stu_resource.items(), key= lambda k:k[1], reverse=True)

## 模型训练

In [ ]:
all_re = []
for top_n in tqdm(N_LIST):
    all_stu_rec_video = {}
    for one_stu in stu_id_index:
        all_stu_rec_video[one_stu] = all_stu_sim_resource[one_stu][:top_n]
    all_recall = []
    all_pre = []
    all_f1 = []
    for one_stu in stu_id_index:
        one_train_video = list(stu_resource[one_stu].keys())
        one_hit = 0
        for one in all_stu_rec_video[one_stu]:
            if one[0] in one_train_video:
                one_hit += 1
        one_recall = one_hit/len(one_train_video)
        all_recall.append(one_recall)
        if len(all_stu_rec_video[one_stu])==0:
            print(one_stu)
        one_pre = one_hit/len(all_stu_rec_video[one_stu])
        all_pre.append(one_pre)
        if (one_recall+one_pre)==0:
            one_f1 = 0
        else:
            one_f1 = (2*one_recall*one_pre)/(one_recall+one_pre)
        all_f1.append(one_f1)
    all_recall_1 = np.mean(all_recall)
    all_pre_1 = np.mean(all_pre)
    all_f1_1 = np.mean(all_f1)
    one_re = {'Top-N':top_n,'Precision':all_pre_1,'Recall':all_recall_1,'F1':all_f1_1,}
    all_re.append(one_re)
all_re_set = pd.DataFrame(all_re)
all_re_set.index = list(all_re_set['Top-N'])
all_re_set.to_csv(f'log/lrr/K.{K}-LBP.{LBP_LEN}-TF.{USE_TF_IDF}-MODULE.{USE_MODULE_INFO}-CLUSTER.{USE_CLUSTER}.csv',index=False)
all_re_set.head(20)

# 抽取学习路径规划信息

In [ ]:
def get_video_list(stu_id):
    one_stu_video = all_stu_rec_video[stu_id]
    need_video = []
    get_video_num = 0
    for one_v in one_stu_video:
        one_video = dict(need_resource.loc[one_v[0]])
        if one_video['re_type'] =='V' and one_video['course_id']==stu_2_course[stu_id]:
            get_video_num += 1
            need_video.append(one_video)
        if get_video_num == VIDEO_LEN:
            break
    return need_video

In [ ]:
def get_weak_kp(stu_id,week_num):    
    stu_learn_kp_list = STU_KP_LEARN[stu_id]['learned_kp']
    stu_unlearn_kp_list = STU_KP_LEARN[stu_id]['unlearned_kp']
    
    stu_m_kp = STU_KP_master_flat[STU_KP_master_flat['stu_id']==stu_id].copy()
    stu_m_kp.index = stu_m_kp['kp_id']
    # 找出所有的薄弱知识点
    stu_have_weak_kp = list(stu_m_kp[stu_m_kp['master']<KP_WEAK_RATE].index)
    # 找出已学且薄弱的知识
    need_kp_list = list(set(stu_learn_kp_list)&set(stu_have_weak_kp))
    if len(need_kp_list)==0:
        need_kp_list = stu_have_weak_kp[:5]
    # 薄弱知识点
    kp_weak = [{'id':one,
                'master':stu_m_kp.loc[one]['master'],
                'name':kp_id_2_name[one],
                } for one in need_kp_list]
    #  已掌握知识点
    kp_master_list = list(stu_m_kp[stu_m_kp['master']>=KP_WEAK_RATE].index)
    kp_master_list = list(set(stu_learn_kp_list)&set(kp_master_list))
    kp_master = [{'id':one,
                'master':stu_m_kp.loc[one]['master'],
                'name':kp_id_2_name[one],
                } for one in kp_master_list]
    #  未学习知识点
    kp_unlearned = [{'id':one,
                'master':stu_m_kp.loc[one]['master'],
                'name':kp_id_2_name[one],
                } for one in stu_unlearn_kp_list]
    return kp_weak,kp_master,kp_unlearned

In [ ]:
ALL_STU_PATH_INFO = []
need_stu_info = studentInfo[['stu_id','course_id','video_t_mean']].copy()
re_need_col = ['course_id','re_id','re_type','kp_id','video_t_mean',
          'kp_0','kp_1','kp_2','kp_3','kp_0_id','kp_1_id','kp_2_id','kp_3_id']
need_resource = resource[re_need_col].copy()
need_resource.index = need_resource['re_id'].tolist()
KP_WEAK_RATE = 0.999
VIDEO_LEN = 10
for one_stu_id in tqdm(risk_stu_have_q):
    # 首先定位危险周数
    one_stu_risk = dict(need_stu_info.loc[one_stu_id]).copy()
    week_num = stu_risk_week_num[one_stu_id]
    one_stu_risk['risk_week'] = week_num
    # 加入课程信息
    one_course = course.loc[one_stu_risk['course_id']]
    one_stu_risk['course_name'] = one_course['name']
    one_stu_risk['course_about'] = one_course['about']
    one_stu_risk['kp_weak_list'],one_stu_risk['kp_master_list'],one_stu_risk['kp_unlearned_list'] = get_weak_kp(one_stu_id,week_num)
    one_stu_risk['video_list_rec'] = get_video_list(one_stu_id)
    ALL_STU_PATH_INFO.append(one_stu_risk)

In [ ]:
with open('path_data/real_data.txt','w',encoding='utf-8') as f:
    f.write(str(ALL_STU_PATH_INFO))

In [ ]:
with open('real_data.txt','r',encoding='utf-8') as f:
    real_data = f.read()
eval(real_data)